In [1]:
import pandas as pd
from sklearn import linear_model
import numpy as np
import os
from mvpa2.suite import *
import glob

/home/language/laugig/.conda/envs/env2.7/lib/python2.7/site-packages/mvpa2/base/hdf5.py:40: H5pyDeprecationWarning: The h5py.highlevel module is deprecated, code should import directly from h5py, e.g. 'from h5py import File'.
  import h5py.highlevel  # >= 2.8.0, https://github.com/h5py/h5py/issues/1063
/home/language/laugig/.conda/envs/env2.7/lib/python2.7/site-packages/mvpa2/testing/tools.py:81: DeprecationWarning: Importing from numpy.testing.decorators is deprecated since numpy 1.15.0, import from numpy.testing instead.
  from numpy.testing.decorators import skipif


In [86]:
from scipy.stats import zscore

In [2]:
spmdir = '/project/3011226.01/bids_DPS/derivatives/SPM/firstlevel'
s=1
subj = str(s).zfill(2)
subjdir = 'sub-0'+subj

In [3]:
predictors = pd.DataFrame()
for run in [1,2,3,4]:
    temp = pd.read_csv('/project/3011226.01/bids_DPS/derivatives/SPM/firstlevel/sub-001/cats'+str(run)+'.txt',sep=',')
    temp['run'] = run
    predictors = predictors.append(temp)
    
predictors['Verb'] = [-1 if x == 0 else 1 for x in predictors.Var3]
predictors = predictors.drop(columns=['Var3','Var4'])
predictors = predictors.rename(columns={"Var1":"S","Var2":"M"})

In [4]:
predictors

,S,M,run,Verb
0,1,1,1,-1
1,1,0,1,1
2,1,0,1,-1
3,1,0,1,1
4,1,0,1,-1
5,1,1,1,-1
6,1,1,1,-1
7,1,1,1,-1
8,1,1,1,1
9,0,1,1,-1


In [10]:
sentence = pd.DataFrame()
for run in [1,2,3,4]:
    temp = pd.read_csv('/project/3011226.01/bids_DPS/derivatives/SPM/firstlevel/sub-001/alltrials'+str(run)+'.csv')
    temp['run'] = run
    sentence = sentence.append(temp)

# remove non-stimuli trials
sentence = sentence[sentence.Condition!='ERR']
sentence = sentence[sentence.Condition!='Question']
sentence = sentence[sentence.Condition!='Embedded']
sentence = sentence[sentence.Condition!='Coordinated']

In [11]:
sentence.head()

,Item_marker,cat_nounAg,cat_nounPa,cat_verb,det_active,det_passive,noun1,noun2,nounAgent,verb,...,verb1acc,verb2said,verb2acc,Voicesaid,VoiceAcc,PassiveSwap,cat_agents,cat_patients,cat_verb1,Cat
0,196,S,M,P,de,de,schaatser,zanger,schaatser,opmerken,...,0,op,0,Active,0,0,S,M,P,SMP
2,17,M,M,C,de,de,cellist,drummer,drummer,wegduwen,...,0,weg,0,Passive,0,0,M,M,C,MMC
4,96,M,M,P,de,de,saxofonist,cellist,saxofonist,bekijken,...,0,0,0,Active,0,0,M,M,P,MMP
5,145,M,M,C,de,de,pianist,muzikant,muzikant,krabben,...,0,0,0,Passive,0,0,M,M,C,MMC
6,193,M,M,P,de,de,muzikant,violist,violist,opmerken,...,0,op,0,Passive,0,0,M,M,P,MMP


In [12]:
M = ['cellist','drummer','gitarist','muzikant','pianist','saxofonist','violist','zanger']
S = ['atleet','bokser','schaatser','surfer','turner','voetballer','wielrenner','zwemmer']
sentence['cat_noun1'] = ["S" if noun in S else "M" for noun in sentence.noun1]
sentence['cat_noun2'] = ["S" if noun in S else "M" for noun in sentence.noun2]

In [13]:
#predictors = predictors.drop(columns=['S','M'])
predictors['run1'] = [1 if run == 1 else 0 for run in predictors.run]
predictors['run2'] = [1 if run == 2 else 0 for run in predictors.run]
predictors['run3'] = [1 if run == 3 else 0 for run in predictors.run]
predictors['run4'] = [1 if run == 4 else 0 for run in predictors.run]
predictors = predictors.drop(columns=['run'])

In [14]:
# thematic roles + syntactic roles + verb
predictors['agent'] = [1 if n == "S" else -1 for n in sentence.cat_agents]
predictors['patient'] = [1 if n == "S" else -1 for n in sentence.cat_patients]
predictors['noun1'] = [1 if n == "S" else -1 for n in sentence.cat_noun1]
predictors['noun2'] = [1 if n == "S" else -1 for n in sentence.cat_noun2]

In [19]:
# thematic roles + syntactic roles + verb in 1 0 form
predictors['agentS'] = [1 if n == "S" else 0 for n in sentence.cat_agents]
predictors['patientS'] = [1 if n == "S" else 0 for n in sentence.cat_patients]
predictors['noun1S'] = [1 if n == "S" else 0 for n in sentence.cat_noun1]
predictors['noun2S'] = [1 if n == "S" else 0 for n in sentence.cat_noun2]
predictors['agentM'] = [1 if n == "M" else 0 for n in sentence.cat_agents]
predictors['patientM'] = [1 if n == "M" else 0 for n in sentence.cat_patients]
predictors['noun1M'] = [1 if n == "M" else 0 for n in sentence.cat_noun1]
predictors['noun2M'] = [1 if n == "M" else 0 for n in sentence.cat_noun2]
predictors['verbC'] = [1 if n == "C" else 0 for n in sentence.cat_verb1]
predictors['verbP'] = [1 if n == "P" else 0 for n in sentence.cat_verb1]
predictors

,S,M,run1,run2,run3,run4,agentS,patientS,noun1S,noun2S,agentM,patientM,noun1M,noun2M,verbC,verbP
0,1,1,1,0,0,0,1,0,1,0,0,1,0,1,0,1
1,1,0,1,0,0,0,0,0,0,0,1,1,1,1,1,0
2,1,0,1,0,0,0,0,0,0,0,1,1,1,1,0,1
3,1,0,1,0,0,0,0,0,0,0,1,1,1,1,1,0
4,1,0,1,0,0,0,0,0,0,0,1,1,1,1,0,1
5,1,1,1,0,0,0,0,1,1,0,1,0,0,1,0,1
6,1,1,1,0,0,0,0,1,0,1,1,0,1,0,0,1
7,1,1,1,0,0,0,1,0,0,1,0,1,1,0,0,1
8,1,1,1,0,0,0,0,1,0,1,1,0,1,0,1,0
9,0,1,1,0,0,0,1,1,1,1,0,0,0,0,0,1


In [20]:
predictors = predictors.drop(columns=['agent'])
predictors = predictors.drop(columns=['patient'])
predictors = predictors.drop(columns=['noun1'])
predictors = predictors.drop(columns=['noun2'])
predictors = predictors.drop(columns=['Verb'])
predictors = predictors.drop(columns=['Verb'])
predictors = predictors.drop(columns=['run'])

KeyError: "['agent'] not found in axis"

In [21]:
predictors

,S,M,run1,run2,run3,run4,agentS,patientS,noun1S,noun2S,agentM,patientM,noun1M,noun2M,verbC,verbP
0,1,1,1,0,0,0,1,0,1,0,0,1,0,1,0,1
1,1,0,1,0,0,0,0,0,0,0,1,1,1,1,1,0
2,1,0,1,0,0,0,0,0,0,0,1,1,1,1,0,1
3,1,0,1,0,0,0,0,0,0,0,1,1,1,1,1,0
4,1,0,1,0,0,0,0,0,0,0,1,1,1,1,0,1
5,1,1,1,0,0,0,0,1,1,0,1,0,0,1,0,1
6,1,1,1,0,0,0,0,1,0,1,1,0,1,0,0,1
7,1,1,1,0,0,0,1,0,0,1,0,1,1,0,0,1
8,1,1,1,0,0,0,0,1,0,1,1,0,1,0,1,0
9,0,1,1,0,0,0,1,1,1,1,0,0,0,0,0,1


In [152]:
run_datasets = []
mask_fname = '/project/3011226.01/bids_DPS/derivatives/ROIs/LRamPFC.nii'
sess = "mb8_single-sents_plan"
for run_id in range(1,(len(sentence[sentence.run==1]))*4+1,4):
    
    spmT = str(run_id).zfill(4)
    bold_fname = os.path.join(spmdir, subjdir, sess, 'con_'+spmT+'.nii')
    mask_fname = os.path.join(spmdir,subjdir,sess, 'mask.nii')

    run_ds = fmri_dataset(samples = bold_fname, chunks = 1, mask=mask_fname)
    run_datasets.append(run_ds)
    
for run_id in range(2,(len(sentence[sentence.run==2]))*4+1,4):
    
    spmT = str(run_id).zfill(4)
    bold_fname = os.path.join(spmdir, subjdir, sess, 'con_'+spmT+'.nii')
    mask_fname = os.path.join(spmdir,subjdir,sess, 'mask.nii')

    run_ds = fmri_dataset(samples = bold_fname, chunks = 2, mask=mask_fname)
    run_datasets.append(run_ds)

for run_id in range(3,(len(sentence[sentence.run==3]))*4+1,4):

    spmT = str(run_id).zfill(4)
    bold_fname = os.path.join(spmdir, subjdir, sess, 'con_'+spmT+'.nii')
    mask_fname = os.path.join(spmdir,subjdir,sess, 'mask.nii')

    run_ds = fmri_dataset(samples = bold_fname, chunks = 3, mask=mask_fname)
    run_datasets.append(run_ds)
    
for run_id in range(4,(len(sentence[sentence.run==4]))*4+1,4):
    
    spmT = str(run_id).zfill(4)
    bold_fname = os.path.join(spmdir, subjdir, sess, 'con_'+spmT+'.nii')
    mask_fname = os.path.join(spmdir,subjdir,sess, 'mask.nii')

    run_ds = fmri_dataset(samples = bold_fname, chunks = 4, mask=mask_fname)
    run_datasets.append(run_ds)

In [153]:
ds = vstack(run_datasets, a=0)
ds = remove_nonfinite_features(ds)

In [154]:
# regression for each cross-validation fold
predall=np.empty(shape=(0,ds.shape[1]+1))
for run in [1,2,3,4]:
    X = predictors[(sentence.run!=run).tolist()]
    #predicted = np.empty(shape=(len(predictors[predictors.run==run]),ds.shape[1]+1))
    #for voxel in range(0,ds.shape[1]):
    observed = ds.samples[sentence.run!=run]
    regr = linear_model.LinearRegression()
    regr.fit(X, observed)
    predicted = regr.predict(predictors[(sentence.run==run).tolist()])
    runlist = np.full((predicted.shape[0],1),run)
    predicted = np.append(predicted,runlist,axis=1)
    predall = np.append(predall,predicted,axis=0)
predy = pd.DataFrame(predall)   

In [ ]:
#X = predictors[predictors.run!=run]
#predicted2 = np.empty(shape=(len(predictors[predictors.run==run]),ds.shape[1]))
#for voxel in range(0,ds.shape[1]):
#    observed = ds.samples[predictors.run!=run,voxel]
#    regr = linear_model.LinearRegression()
#    regr.fit(X, observed)
#    predicted2[:,voxel] = regr.predict(predictors[predictors.run==run])

In [77]:
for run in [1,2,3,4]:
    predrun = predy[predy.iloc[:,-1]==run].iloc[:,0:-1].to_numpy()
    observed = ds.samples[sentence.run==run]
    vox_diff = np.empty(shape=(ds.shape[1],4))
    for voxel in range(0,ds.shape[1]):
        diffmat=np.empty(shape=(len(predrun),len(predrun)))
        for pred in range(0,len(predrun)):
            for obs in range(0,len(observed)):
                # squared differences (errors) of predictions for each voxel
                diffmat[pred,obs]= (predrun[pred,voxel] - observed[obs,voxel])**2
        diff1 = np.reshape(diffmat,(1,len(predrun)*len(predrun)))
        # z-score entire matrix
        diff2 = (diff1 - diff1.mean())/diff1.std(ddof=1)
        diff3 = np.reshape(diff2,(len(predrun),len(predrun)))
        correct = np.mean(np.diagonal(diff3))
        incorrect = (sum(sum(diff3))-sum(np.diagonal(diff3)))/((len(predrun)*len(predrun))-len(predrun))
        vox_diff[voxel,run-1] = correct-incorrect

In [105]:
for run in [1,2,3,4]:
    predrun = predy[predy.iloc[:,-1]==run].iloc[:,0:-1].to_numpy()
    observed = ds.samples[sentence.run==run]
    vox_diff = np.empty(shape=(ds.shape[1],4))
    for voxel in range(0,ds.shape[1]):
        diffmat=np.empty(shape=(len(predrun),len(predrun)))
        for pred in range(0,len(predrun)):
            for obs in range(0,len(observed)):
                # squared differences (errors) of predictions for each voxel
                diffmat[pred,obs]= (predrun[pred,voxel] - observed[obs,voxel])**2
        # z-score entire matrix
        z_diff = zscore(diffmat)
        correct = np.mean(np.diagonal(z_diff))
        incorrect = (sum(sum(z_diff))-sum(np.diagonal(z_diff)))/((len(predrun)*len(predrun))-len(predrun))
        vox_diff[voxel,run-1] = correct-incorrect

In [94]:
z_diff = zscore(diffmat,axis=None)

In [106]:
voxmean = np.mean(vox_diff,axis=1)*-1

In [107]:
map2nifti(ds, voxmean).to_filename('/project/3011226.01/bids_DPS/derivatives/encoding_model/them_subj_verb_con.nii.gz') 

## Testing by classification of random sent

In [155]:
# regression for each cross-validation fold
predall=np.empty(shape=(0,ds.shape[1]+1))
for run in [1,2,3,4]:
    X = predictors[(sentence.run!=run).tolist()]
    #predicted = np.empty(shape=(len(predictors[predictors.run==run]),ds.shape[1]+1))
    #for voxel in range(0,ds.shape[1]):
    observed = ds.samples[sentence.run!=run]
    regr = linear_model.LinearRegression()
    regr.fit(X, observed)
    predicted = regr.predict(predictors[(sentence.run==run).tolist()])
    runlist = np.full((predicted.shape[0],1),run)
    predicted = np.append(predicted,runlist,axis=1)
    predall = np.append(predall,predicted,axis=0)
predy = pd.DataFrame(predall)   

In [132]:
pred = predictors.to_numpy()


In [133]:
np.take(pred,np.random.permutation(pred.shape[0]),axis=0,out=pred)

array([[0, 1, 0, ..., 0, 1, 0],
       [0, 1, 0, ..., 0, 0, 1],
       [1, 1, 0, ..., 0, 1, 0],
       ...,
       [1, 1, 1, ..., 1, 0, 1],
       [1, 1, 0, ..., 1, 1, 0],
       [1, 0, 1, ..., 1, 0, 1]])

In [156]:
# regression for each cross-validation fold
predall_rand=np.empty(shape=(0,ds.shape[1]+1))
for run in [1,2,3,4]:
    X = predictors[(sentence.run!=run).tolist()]
    #predicted = np.empty(shape=(len(predictors[predictors.run==run]),ds.shape[1]+1))
    #for voxel in range(0,ds.shape[1]):
    observed = ds.samples[sentence.run!=run]
    regr = linear_model.LinearRegression()
    regr.fit(X, observed)
    pred = predictors[(sentence.run==run).tolist()].to_numpy()
    pred_shuf = np.take(pred,np.random.permutation(pred.shape[0]),axis=0,out=pred)
    predicted_rand = regr.predict(pred_shuf)
    runlist_rand = np.full((predicted_rand.shape[0],1),run)
    predicted_rand = np.append(predicted_rand,runlist_rand,axis=1)
    predall_rand = np.append(predall_rand,predicted_rand,axis=0)
predy_rand = pd.DataFrame(predall_rand) 

In [120]:
from scipy.spatial import distance

In [136]:
distance.euclidean(predy.iloc[0,:-1].to_numpy(),observed[0,:])

991.5193502430906

In [137]:
distance.euclidean(predy_rand.iloc[0,:-1].to_numpy(),observed[0,:])


1033.4988767498905

In [157]:
class_acc = []
observed = ds.samples
for sent in range(0,123):
    dist_cor = distance.euclidean(predy.iloc[sent,:-1].to_numpy(),observed[sent,:])
    dist_incor = distance.euclidean(predy_rand.iloc[sent,:-1].to_numpy(),observed[sent,:])
    class_acc.append(1 if dist_cor < dist_incor else 0)

In [158]:
sum(class_acc)

57

In [159]:
len(class_acc)

123